# Lab 1: Azure Functions Integration with AI Foundry Agent

This notebook demonstrates how to integrate Azure Functions as tools within an AI Foundry agent.

## Learning Objectives
1. Configure Azure Functions client
2. Create and invoke Azure Functions
3. Register functions as agent tools
4. Handle responses and errors

## Prerequisites
- Azure subscription
- Azure Function App deployed
- Python 3.10 or later
- Required packages installed (`pip install -r requirements.txt`)

## Step 1: Setup and Imports

First, let's import the necessary modules and set up logging.

In [ ]:
import logging
import os
from dotenv import load_dotenv

from src.abstractions.azure_functions import (
    FunctionConfig,
    AzureFunctionsClient,
    DataProcessorFunction,
    IntegrationFunction
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

logger.info("Environment setup complete")

## Step 2: Configure Azure Function Connection

Create a configuration object for your Azure Function. You can use either:
- **Function Key**: For development and testing
- **Managed Identity**: For production deployments

In [ ]:
# Option 1: Using Function Key (recommended for development)
function_config = FunctionConfig(
    function_url=os.getenv("AZURE_FUNCTION_URL", "https://your-app.azurewebsites.net/api/your-function"),
    function_key=os.getenv("AZURE_FUNCTION_KEY", "your-function-key-here"),
    timeout=30
)

# Option 2: Using Managed Identity (recommended for production)
# function_config = FunctionConfig(
#     function_url=os.getenv("AZURE_FUNCTION_URL"),
#     use_managed_identity=True,
#     timeout=30
# )

logger.info(f"Function configured: {function_config.function_url}")

## Step 3: Create Azure Functions Client

Initialize the client that will handle communication with your Azure Function.

In [ ]:
# Create the client
functions_client = AzureFunctionsClient(function_config)

logger.info("Azure Functions client initialized")

## Step 4: Invoke Azure Function (Synchronous)

Make a synchronous call to your Azure Function.

In [ ]:
# Prepare payload
payload = {
    "operation": "process",
    "data": {
        "values": [1, 2, 3, 4, 5],
        "transform": "sum"
    }
}

# Invoke function
try:
    result = functions_client.invoke_function(payload)
    logger.info(f"Function result: {result}")
    print("✅ Function invoked successfully!")
    print(f"Result: {result}")
except Exception as e:
    logger.error(f"Function invocation failed: {e}")
    print(f"❌ Error: {e}")

## Step 5: Invoke Azure Function (Asynchronous)

For better performance with multiple calls, use the async interface.

In [ ]:
import asyncio

async def invoke_async_function():
    """Demonstrate async function invocation."""
    payload = {
        "operation": "process",
        "data": {"message": "Hello from async!"}
    }
    
    try:
        result = await functions_client.invoke_function_async(payload)
        logger.info(f"Async result: {result}")
        print("✅ Async function invoked successfully!")
        print(f"Result: {result}")
        return result
    except Exception as e:
        logger.error(f"Async invocation failed: {e}")
        print(f"❌ Error: {e}")
        return None

# Run async function
await invoke_async_function()

## Step 6: Use Pre-built Function Tools

The package includes pre-built function wrappers for common scenarios.

In [ ]:
# Data Processor Function
data_processor = DataProcessorFunction(function_config)

# Process some data
data = {
    "values": [10, 20, 30, 40, 50],
    "operation": "average"
}

try:
    result = data_processor.process_data(data)
    print(f"✅ Data processed: {result}")
except Exception as e:
    print(f"❌ Processing error: {e}")

In [ ]:
# Integration Function
integration_function = IntegrationFunction(function_config)

# Call external service
try:
    result = integration_function.call_external_service(
        service="weather-api",
        params={"location": "Chicago", "units": "metric"}
    )
    print(f"✅ External service called: {result}")
except Exception as e:
    print(f"❌ Integration error: {e}")

## Step 7: Error Handling and Best Practices

Demonstrate proper error handling and logging.

In [ ]:
def safe_function_call(client: AzureFunctionsClient, payload: dict) -> dict:
    """Safely invoke a function with comprehensive error handling."""
    try:
        logger.info("Attempting function invocation")
        result = client.invoke_function(payload)
        logger.info("Function invocation successful")
        return result
    except requests.exceptions.Timeout:
        logger.error("Function timeout - consider increasing timeout value")
        return {"error": "timeout", "message": "Function took too long to respond"}
    except requests.exceptions.HTTPError as e:
        logger.error(f"HTTP error: {e.response.status_code}")
        return {"error": "http_error", "status_code": e.response.status_code}
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        return {"error": "unknown", "message": str(e)}

# Test error handling
result = safe_function_call(functions_client, {"test": "data"})
print(f"Result: {result}")

## 🎯 Copilot Tips

### GitHub Copilot Chat Commands
- `/explain` - Get explanation of Azure Functions code
- `/fix` - Fix errors in function calls
- `/tests` - Generate tests for your functions
- `/doc` - Generate docstrings

### Useful Prompts
- "Create a new Azure Function that processes JSON data"
- "Add retry logic to this function call"
- "Generate unit tests for the AzureFunctionsClient"
- "Add error handling for network failures"

## 📚 Additional Resources

- [Azure Functions Documentation](https://learn.microsoft.com/en-us/azure/azure-functions/)
- [Azure AI Foundry](https://azure.microsoft.com/en-us/products/ai-studio/)
- [Python Async/Await Guide](https://docs.python.org/3/library/asyncio.html)

## ✅ Summary

In this lab, you learned:
- ✅ How to configure Azure Functions clients
- ✅ Synchronous and asynchronous function invocation
- ✅ Using pre-built function tools
- ✅ Error handling best practices
- ✅ Integration with AI agents

**Next**: Proceed to Lab 2 to learn about Logic Apps integration!